In [1]:
# !pip uninstall snowflake

In [2]:
# !pip uninstall snowflake -y


In [3]:
!pip install langchain pymongo bs4 openai tiktoken gradio requests lxml argparse unstructured

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pydantic_core-2.23.4-cp312-none-win_amd64.whl.metadata (6.7 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp312-none-win_amd64.whl (1.9 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.6
    Uninstalling pydantic-2.10.6:
      Successfully uninstalled pydantic-2.10.6


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydantic-ai-slim 0.0.19 requires pydantic>=2.10, but you have pydantic 2.9.2 which is incompatible.
pydantic-graph 0.0.19 requires pydantic>=2.10, but you have pydantic 2.9.2 which is incompatible.
scrapegraph-py 1.10.2 requires pydantic>=2.10.2, but you have pydantic 2.9.2 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [4]:
!pip install snowflake-connector-python

In [5]:
!pip install crewai[tools]
!pip install crewai[embeddings]

  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl (2.0 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.23.4
    Uninstalling pydantic_core-2.23.4:
      Successfully uninstalled pydantic_core-2.23.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
unstructured-client 0.27.0 requires pydantic<2.10.0,>=2.9.2, but you have pydantic 2.10.6 which is incompatible.


In [6]:
!pip install --upgrade pip setuptools wheel


  Using cached pip-25.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached pip-25.0-py3-none-any.whl (1.8 MB)
Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)


ERROR: To modify pip, please run the following command:
C:\Users\sarfarazuddin.s\AppData\Local\anaconda3\python.exe -m pip install --upgrade pip setuptools wheel


In [7]:
!pip install chroma-hnswlib --only-binary=:all:

In [8]:
!pip install crewai-tools crewai langchain_openai python-dotenv

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
from crewai import Agent, Crew, Task, Process
import os
from langchain_openai import ChatOpenAI

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

In [17]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, SeleniumScrapingTool

In [18]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://en.wikipedia.org/wiki/Medicine"
)

# Agents

In [21]:
Writer = Agent(
    role="Senior Medical content writer",
    goal="Be the most friendly and helpful "
         "guide to your students",
    backstory=(
        "You are best Medical content writer "
        "are now working as senior Medical writer and "
        "support to students for their career."
        "Make sure to provide full complete answers, "
        "and make no assumptions."
    ),
    llm=llm,
    # tools=[docs_scrape_tool],
    allow_delegation=False,
    verbose=True
)

In [22]:
quality_agent = Agent(
    role="Quality Content reviewer",
    goal="Get recognition for providing the "
         "best support quality assurance to the students",
    backstory=(
        "You are best quality content reviewer "
        "now working as quality content reviewer "
        "on a request from student ensuring that "
        "the Senior Medical content writer is "
        "providing the best guidance possible."
        "You need to make sure that the Senior Medical content writer "
        "is providing full complete answers, and make no assumptions."
    ),
    llm=llm,
    verbose=True
)

# Tasks

In [23]:
resolution = Task(
    description=(
        "student just reached out with a super important question {question} "
        "Make sure to use everything you know "
        "to provide the best support possible."
        "You must strive to provide a complete "
        "and accurate response to the student question {question}."
        "If the question {question} is not related to Medical don't give answer to that question."
    ),
    expected_output=(
        "A detailed informative response to the "
        "student's question {question} and addresses "
        "all aspects of their question."
        "Dont give answers other than Medical."
    ),
    tools=[docs_scrape_tool],
    agent=Writer,
)

In [24]:
quality_review = Task(
    description=(
        "Review the response drafted by the Senior Medical content writer for student's question {question}."
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
        "high-quality standards expected for student support."
        "Check if Senior Medical content writer is giving answers other than Medical related question {question}."
        "Verify that all parts of the student's question {question} have been addressed "
        "thoroughly, with a helpful and friendly tone."
        "Check for references and sources used to "
        "find the information."
        "If the question {question} is not related to Medical don't give answer to that question."
        
        
        
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the student."
        "This response should fully address the "
        "student's question {question}, incorporating all "
        "relevant feedback and improvements."
        "If the question {question} is not related to Medical don't give answer to that question."
    ),
    agent=quality_agent
)

In [25]:
crew=Crew(
    agents=[Writer, quality_agent],
    tasks=[resolution, quality_review],
    verbose=1,
    memory=True,
)

In [26]:
inputs = {
    "question": "Clinical examination involves the study of"
}

result = crew.kickoff(inputs=inputs)

# Agent: Senior Medical content writer
## Task: student just reached out with a super important question Clinical examination involves the study of Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the student question Clinical examination involves the study of.If the question Clinical examination involves the study of is not related to Medical don't give answer to that question.


# Agent: Senior Medical content writer
## Thought: I need to gather detailed information about clinical examination to provide a comprehensive answer to the student's question.
## Using tool: [Read website content]
## Tool Input: 
"{}"
## Tool Output: 

Medicine - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page Contents Current events Random article About Wikipedia Contact us
Contribute
Help Learn to edit Community portal Recent changes Upload file
Search
Search
Appearance
Donate
Cre

In [27]:
from IPython.display import Markdown
Markdown(result.raw)

Thank you for your question regarding what clinical examination involves. Clinical examination is a vital process in the medical field that allows healthcare providers to assess a patient's health comprehensively. It encompasses several key components, which I will outline below for clarity and thorough understanding.

### 1. Medical History
The first step in a clinical examination is gathering the patient's medical history, which includes:

- **Chief Complaint**: The primary reason the patient is seeking medical attention.
- **Current Activity**: Information about the patient's daily activities and any limitations they may be experiencing.
- **Family History**: A record of health issues in the patient's family that may be relevant to their current health status.
- **History of Present Illness**: Detailed information about the symptoms the patient is experiencing, including onset, duration, and characteristics.
- **Medications**: A comprehensive list of current medications, including dosages and duration of use.
- **Past Medical History**: Any previous illnesses, surgeries, or significant health events that could impact the patient's current health.
- **Review of Systems**: A systematic inquiry into various body systems to identify any additional symptoms that may not have been initially reported.
- **Social History**: Information about the patient's lifestyle, including occupation, substance use, and living conditions, which can influence health.

### 2. Physical Examination
Following the medical history, a physical examination is conducted, which involves a hands-on assessment of the patient. Key techniques include:

- **Inspection**: Observing the patient for any visible abnormalities or signs of distress.
- **Palpation**: Using touch to feel for abnormalities in the body, such as lumps or areas of tenderness.
- **Percussion**: Tapping on specific areas of the body to assess underlying structures and identify any abnormalities.
- **Auscultation**: Listening to the sounds produced by internal organs, typically using a stethoscope, to detect any irregularities.

### 3. Vital Signs
Vital signs are essential physiological parameters that are measured during the clinical examination, including:

- Height and weight
- Body temperature
- Blood pressure
- Pulse rate
- Respiration rate
- Hemoglobin oxygen saturation

### 4. Specific Areas of Examination
Depending on the patient's medical history and presenting symptoms, the clinical examination may focus on specific areas, such as:

- Abdomen and rectum
- Cardiovascular system (heart and blood vessels)
- General appearance (assessing nutritional status, jaundice, pallor, clubbing)
- Genitalia (including pregnancy status)
- Head, eyes, ears, nose, and throat (HEENT)
- Musculoskeletal system (spine and extremities)
- Neurological system (assessing consciousness, awareness, and brain function)
- Psychiatric evaluation (evaluating mental state and mood)
- Respiratory system (lungs and airways)
- Skin examination

### Conclusion
The clinical examination is a critical component of the diagnostic process, enabling healthcare providers to gather essential information that aids in formulating a treatment plan or determining the need for further diagnostic testing. It is essential for ensuring comprehensive patient care and effective management of health conditions.

For further reading and reference, I recommend consulting medical textbooks on clinical examination or guidelines from reputable medical organizations such as the American Academy of Family Physicians (AAFP) or the British Medical Journal (BMJ).

If you have any further questions or need clarification on any specific aspect of the clinical examination, feel free to ask! I'm here to help you understand this important topic better.

In [28]:
inputs = {
    "question": "What is Data Science"
}

result = crew.kickoff(inputs=inputs)

# Agent: Senior Medical content writer
## Task: student just reached out with a super important question What is Data Science Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the student question What is Data Science.If the question What is Data Science is not related to Medical don't give answer to that question.


# Agent: Senior Medical content writer
## Thought: I need to determine if the question "What is Data Science?" has any relevance to the field of medicine, as I am only to provide answers related to medical content.
## Using tool: [Read website content]
## Tool Input: 
"{}"
## Tool Output: 

Medicine - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page Contents Current events Random article About Wikipedia Contact us
Contribute
Help Learn to edit Community portal Recent changes Upload file
Search
Search
Appearance
Donate
Create account
Log in
Personal 

In [29]:
from IPython.display import Markdown
Markdown(result.raw)

The question "What is Data Science?" is not directly related to the field of medicine, which focuses on the science and practice of caring for patients, managing diagnosis, treatment, and prevention of diseases. Therefore, I will not provide an answer to this question as it falls outside the medical context. If you have any questions related to medicine or healthcare, please feel free to ask!

In [30]:
# import gradio as gr

# # Create a web interface for the app using Gradio
# with gr.Blocks(title="Question Answering Application") as demo:
#     gr.Markdown("# Question Answering Application")
    
#     # Input textbox for the user to enter their question
#     textbox = gr.Textbox(label="Enter your Question:", placeholder="Type your question here...")
    
#     with gr.Row():
#         # Button to trigger processing
#         button = gr.Button("Submit", variant="primary")
    
#     with gr.Column():
#         # Single textbox to display the final output
#         output = gr.Textbox(lines=2, max_lines=10, label="Response:")

#     # Define button behavior without using functions
#     button.click(
#         lambda question: crew.kickoff(inputs={"question": question}).raw,  # Extract only the final output
#         inputs=textbox,
#         outputs=output
#     )

# # Launch the Gradio app
# demo.launch()
